In [22]:
import random
import pandas as pd
import numpy as np
import scipy as sp
import nltk
import PyPDF2

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge

In [2]:
# Import the data to a df
df = pd.read_csv('data/siop_ml_train_participant.csv')

# Import the test data to a new df
eval_df = pd.read_csv('data/siop_ml_dev_participant.csv')

# Import the weighting matrix
# weights = pd.read_csv('data/weights/mvp_matrix.csv')
weights = pd.read_csv('data/weights/dummy_matrix_no_impact.csv')

# Since we want to select a specific index later for the weights, we'll set the df index here
weighting_matrix = weights.set_index('sj_item_name')

# Confirm that the data has been imported and is formatted correctly
df.head(3)

,Respondent_ID,open_ended_1,open_ended_2,open_ended_3,open_ended_4,open_ended_5,E_Scale_score,A_Scale_score,O_Scale_score,C_Scale_score,N_Scale_score
0,10446116527,"I would change my vacation week, because I am ...",I would reach out to my boss and ask him or he...,I would not go. I am a not a social person. I ...,I would ask my manager why he/she gave me such...,I would find this experience super enjoyable. ...,2.250000,3.750000,3.166667,3.750000,2.916667
1,10440100535,I would talk to my colleague and see if they w...,I would continue to work on the project that w...,I would talk to my colleague and try to talk t...,I would feel upset about the negative feedback...,I would find this experience enjoyable. I feel...,4.666667,4.416667,4.583333,5.000000,1.333333
2,10462850071,I would feel upset because perhaps I already b...,I would start working on the project now and g...,I would feel guilty about thinking about not g...,I would feel really defensive about it. I woul...,I would find it enjoyable because I would be r...,2.250000,4.750000,4.083333,4.666667,2.166667


Sooo the function below does the past and present tense, but it looks like there is no variation in these questions, they all come up as one.  Unless I've somehow done it wrong.

In [56]:
def tenseTag(variable):
    for row in variable:
        if "VBD" or "VBN" in nltk.pos_tag(row):
            past = 1
        else:
            past = 0
        
        if "VBG" or "VBP" or "VPZ" in nltk.pos_tag(row):
            present = 1
        else:
            present = 0
    
        df["past"] = past
        df["present"] = present

In [58]:
tenseTag(df["open_ended_1"])
df.head()
counts = pd.value_counts(df["present"]).to_frame().reset_index()
counts = pd.value_counts(df["past"]).to_frame().reset_index()
print(counts)

   index  past
0      1  1088


Word lengths, apparently a good predicter of openness

In [19]:
def wordLengths(variable):
    for row in variable:
        lengths = list(map(len,row.split()))
        sumlengths = sum(i > 5 for i in lengths)
        #print(sumlengths)
        percentoverfive = sumlengths/len(row)*100
        print(percentoverfive)

In [20]:
wordLengths(df["open_ended_1"])


4.135338345864661
3.711340206185567
5.167958656330749
5.660377358490567
4.366812227074235
3.985507246376811
4.716981132075472
2.6881720430107525
5.687203791469194
5.573770491803279
5.048076923076923
3.4220532319391634
4.980842911877394
4.819277108433735
4.66321243523316
4.242424242424243
5.9171597633136095
4.772727272727273
4.529616724738676
4.761904761904762
5.030181086519115
5.0113895216400905
4.794520547945205
4.751619870410368
5.084745762711865
4.477611940298507
3.755868544600939
5.454545454545454
2.941176470588235
3.8834951456310676
4.0625
2.912621359223301
6.61764705882353
7.758620689655173
3.9473684210526314
4.970760233918129
2.877697841726619
5.828220858895705
6.282722513089005
4.835589941972921
4.0
3.75
4.602510460251046
5.194805194805195
5.405405405405405
3.6900369003690034
5.154639175257731
5.454545454545454
5.0
5.555555555555555
5.802047781569966
4.225352112676056
5.627705627705628
4.643962848297214
3.004291845493562
4.216867469879518
4.9504950495049505
3.551912568306011
3.

Negative Affect

In [23]:
file = open('LM_Negative.pdf', 'rb')

# creating a pdf reader object
fileReader = PyPDF2.PdfFileReader(file)

# print the number of pages in pdf file
print(fileReader.numPages)

#number_of_pages = tabula.read_pdf(fileReader)
pageObj = fileReader.getPage(0)
page_content = pageObj.extractText()
print(page_content)

52
Loughran
-McDonald Sentiment Word Lists 
Œ Negative
  ABANDON
 
ABANDONED
 ABANDONING
 
ABANDONMENT
 ABANDONMENTS
 ABANDONS
 ABDICATED
 
ABDICATES
 ABDICATING
 ABDICATION
 ABDICATIONS
 ABERRANT
 ABERRATION
 ABERRATIONAL
 ABERRATIONS
 ABETTING
 ABNORMAL
 ABNORMALITIES
 ABNORMALITY
 
ABNORMALLY
 ABOLISH
 ABOLISHED
 
ABOLISHES
 ABOLISHING
 ABROGATE
 ABROGATED
 
ABROGATES
 ABROGATING
 
ABROGATION
 
ABROGATIONS
 ABRUPT
 ABRUPTLY
 
ABRUPTNESS
 ABSENCE
 ABSENCES
 ABSENTEEISM
 ABUSE
 ABUSED
 
ABUSES
 ABUSING
 
ABUSIVE
 ABUSIVELY
 ABUSIVENESS
 ACCIDENT
 


In [26]:
negativewords = ""
for page in range(fileReader.numPages):
    pageObj = fileReader.getPage(page)
    page_content = pageObj.extractText()
    negativewords = negativewords + page_content
#print(negativewords)
negwords = negativewords.split()

negwords = negwords[7:]
print(negwords)

['ABANDON', 'ABANDONED', 'ABANDONING', 'ABANDONMENT', 'ABANDONMENTS', 'ABANDONS', 'ABDICATED', 'ABDICATES', 'ABDICATING', 'ABDICATION', 'ABDICATIONS', 'ABERRANT', 'ABERRATION', 'ABERRATIONAL', 'ABERRATIONS', 'ABETTING', 'ABNORMAL', 'ABNORMALITIES', 'ABNORMALITY', 'ABNORMALLY', 'ABOLISH', 'ABOLISHED', 'ABOLISHES', 'ABOLISHING', 'ABROGATE', 'ABROGATED', 'ABROGATES', 'ABROGATING', 'ABROGATION', 'ABROGATIONS', 'ABRUPT', 'ABRUPTLY', 'ABRUPTNESS', 'ABSENCE', 'ABSENCES', 'ABSENTEEISM', 'ABUSE', 'ABUSED', 'ABUSES', 'ABUSING', 'ABUSIVE', 'ABUSIVELY', 'ABUSIVENESS', 'ACCIDENT', 'ACCIDENTAL', 'ACCIDENTALLY', 'ACCIDENTS', 'ACCUSATION', 'ACCUSATIONS', 'ACCUSE', 'ACCUSED', 'ACCUSES', 'ACCUSING', 'ACQUIESCE', 'ACQUIESCED', 'ACQUIESCES', 'ACQUIESCING', 'ACQUIT', 'ACQUITS', 'ACQUITTAL', 'ACQUITTALS', 'ACQUITTE', 'D', 'ACQUITTING', 'ADULTERATE', 'ADULTERATED', 'ADULTERATING', 'ADULTERATION', 'ADULTERATIONS', 'ADVERSARIAL', 'ADVERSARIES', 'ADVERSARY', 'ADVERSE', 'ADVERSELY', 'ADVERSITIES', 'ADVERSITY', '

Percentage of negative words: Ok, this one is not working either.  "Negative" seems not to be translating to the dataset correctly.  Any way you could check it out to see what's wrong?

In [54]:
def negAffect(variable):
    for row in variable:
        negative = 0
        words = row.split()
        for word in words:
            if word.upper() in negwords:
                negative = negative + 1
                print(negative)
            percentnegative = negative/len(row)
            df["negative"] = negative
            df["percentneg"] = percentnegative
    print(df.head())

In [55]:
negAffect(df["open_ended_1"])
countsneg = pd.value_counts(df["percentneg"]).to_frame().reset_index()
countsneg1 = pd.value_counts(df["negative"]).to_frame().reset_index()
print(countsneg)
print(countsneg1)

1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
2
3
1
2
1
1
1
1
1
2
1
1
1
2
1
1
1
2
1
2
1
1
2
1
1
1
1
1
2
3
4
5
1
2
1
1
1
2
3
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
2
1
1
1
1
2
3
4
5
1
1
1
1
1
1
1
1
2
1
1
2
3
1
1
2
3
1
1
2
1
1
1
2
1
1
1
1
1
1
2
1
1
1
2
1
2
1
1
1
1
1
1
2
3
1
1
1
1
1
1
1
1
1
1
2
1
2
1
1
1
1
1
1
2
3
1
1
2
1
1
1
1
1
1
2
1
1
2
3
1
1
1
1
2
1
1
1
1
1
1
1
1
2
1
1
1
2
1
1
1
2
1
1
1
2
1
1
1
1
1
1
2
3
1
1
1
2
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
2
1
2
1
1
2
3
1
1
2
1
1
1
1
2
1
1
1
1
1
1
1
2
3
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
2
3
1
1
1
2
1
2
1
1
1
1
2
3
1
1
1
1
2
1
1
1
1
1
2
3
1
2
1
2
1
2
3
1
1
2
1
2
3
1
1
1
1
2
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
2
3
1
1
2
1
1
1
1
2
1
2
1
1
2
1
1
2
3
1
2
1
1
2
1
2
1
2
3
4
1
1
1
1
2
1
1
2
1
2
1
2
1
1
1
1
1
1
2
1
2
1
2
1
1
1
1
1
1
1
2
1
1
1
1
1
2
1
1
2
3
1
2
1
1
1
2
3
1
2
1
1
1
1
1
1
1
1
1
1
1
2
3
4
1
2
1
1
2
1
1
2
3
1
1
1
1
1
1
2
1
2
1
1
1
2
3
1
1
1
1
2
1
1
2
1
1
1
1
2
1
2
1
2
1
1
1
1
2
3
1
1
1
2
